In [2]:
# download dataset from the UCI website
!curl -o uci-labelled-sentences.zip https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84188    0 84188    0     0   174k      0 --:--:-- --:--:-- --:--:--  174k


In [3]:
# unzip dataset in Colab
!unzip uci-labelled-sentences.zip

Archive:  uci-labelled-sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [4]:
!pip install pandas scikit-learn keras tensorflow

In [5]:
!pip install tensorflow

In [6]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping


In [7]:
df_list = []

# Yelp
df_yelp = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_yelp['source'] = 'yelp'
df_list.append(df_yelp)

# Amazon
df_amazon = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', names=['sentence', 'label'], sep='\t')
df_amazon['source'] = 'amazon'
df_list.append(df_amazon)

# IMDB
df_imdb = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', names=['sentence', 'label'], sep='\t')
df_imdb['source'] = 'imdb'
df_list.append(df_imdb)

# Concatenate the dataframes
df = pd.concat(df_list)

df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [8]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['sentence'].values)
X = tokenizer.texts_to_sequences(df['sentence'].values)
X = pad_sequences(X)
y = df['label'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12)

In [10]:
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, 64))
  model.add(LSTM(16))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model()

In [11]:
model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test), callbacks = [EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)])

Epoch 1/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 73s 462ms/step - accuracy: 0.5957 - loss: 0.6732 - val_accuracy: 0.7273 - val_loss: 0.5277
Epoch 2/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 80s 450ms/step - accuracy: 0.8584 - loss: 0.3841 - val_accuracy: 0.8394 - val_loss: 0.3797
Epoch 3/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 82s 451ms/step - accuracy: 0.9434 - loss: 0.1979 - val_accuracy: 0.8394 - val_loss: 0.3931
Epoch 4/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 71s 468ms/step - accuracy: 0.9571 - loss: 0.1402 - val_accuracy: 0.8364 - val_loss: 0.4641
Epoch 4: early stopping


In [13]:
# Save the model in the Keras format
model.save('uci_sentimentanalysis.h5')

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
